## Run Inference on all deployed endpoints: Various combinations of payloads, concurrency levels, model configurations
---------------------
*This notebook works best with the conda_python3 kernel on a ml.t3.medium machine*.

#### This step of our solution design includes running inferences on all deployed model endpoints (with different configurations, concurrency levels and payload sizes). This notebook runs inferences in a manner that is calls endpoints concurrently and asychronously to generate responses and record metrics. Here are some of the key components:

- **Accessing the deployed endpoints**, creating a predictor object for these endpoints to call them during inference time.

- **Functions to define metrics**: This notebook sets stage for metrics to be recorded during the time of invocation of all these models for benchmarking purposes.

- **Running Actual Inferences**: Once the metrics are defined, we set a blocker function that is responsible for creating inference on a single payload called get_inference. We then run a series of asynchronous functions that can be viewed in the code (link above), to create asychronous inferefences on the deployed models. The way we send requests are by creating combinations: this means creating combinations of payloads of different sizes that can be viewed in the config.yml file, with different concurrency levels (in this case we first go through all patches of payloads with a concurrency level of 1, then 2, and then 4). You can set this to your desired value.

### Import all of the necessary libraries below to run this notebook

In [1]:
# if interactive mode is set to no -> pickup fmbench from Python installation path
# if interactive mode is set to yes -> pickup fmbench from the current path (one level above this notebook)
# if interactive mode is not defined -> pickup fmbench from the current path (one level above this notebook)
# the premise is that if run non-interactively then it can only be run through main.py which will set interactive mode to no
import os
import sys
if os.environ.get("INTERACTIVE_MODE_SET", "yes") == "yes":
    sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import glob
import time
import json
import io
import copy
import boto3
import asyncio
import logging
import botocore
import itertools
import sagemaker
import pandas as pd
import importlib.util
from fmbench.utils import *
from fmbench.globals import * 
from datetime import datetime
from datetime import timezone
from transformers import AutoTokenizer
from sagemaker.predictor import Predictor
import importlib.resources as pkg_resources
from sagemaker.serializers import JSONSerializer
from typing import Dict, List, Optional, Tuple, Union
from fmbench.scripts.bedrock_predictor import BedrockPredictor

sagemaker.config INFO - Not applying SDK defaults from location: /opt/homebrew/share/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/madhurpt/Library/Application Support/sagemaker/config.yaml
config file current -> configs/config-claude-models.yml, None
Loaded config: {'general': {'name': 'fmbench-claude', 'model_name': 'claude'}, 'aws': {'region': 'us-east-1', 'sagemaker_execution_role': 'arn:aws:iam::121797993273:user/ab3', 'bucket': 'sagemaker-fmbench-write-121797993273'}, 'dir_paths': {'data_prefix': 'data', 'prompts_prefix': 'prompts', 'all_prompts_file': 'all_prompts.csv', 'metrics_dir': 'metrics', 'models_dir': 'models', 'metadata_dir': 'metadata'}, 's3_read_data': {'read_bucket': 'sagemaker-fmbench-read-121797993273', 'scripts_prefix': 'scripts', 'script_files': ['hf_token.txt'], 'source_data_prefix': 'source_data', 'tokenizer_prefix': 'tokenizer', 'prompt_template_dir': 'prompt_template', 'prompt_template_file': 'prompt_template.tx

No files found in S3 Bucket: 'sagemaker-fmbench-read-121797993273' with Prefix: 'tokenizer'


#### Pygmentize globals.py to view and use any of the globally initialized variables 

In [3]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

#### Load the Config.yml file that contains information that is used across this benchmarking environment

In [4]:
config = load_config(CONFIG_FILE)
logger.info(json.dumps(config, indent=2))

INFO:__main__:{
  "general": {
    "name": "fmbench-claude",
    "model_name": "claude"
  },
  "aws": {
    "region": "us-east-1",
    "sagemaker_execution_role": "arn:aws:iam::121797993273:user/ab3",
    "bucket": "sagemaker-fmbench-write-121797993273"
  },
  "dir_paths": {
    "data_prefix": "data",
    "prompts_prefix": "prompts",
    "all_prompts_file": "all_prompts.csv",
    "metrics_dir": "metrics",
    "models_dir": "models",
    "metadata_dir": "metadata"
  },
  "s3_read_data": {
    "read_bucket": "sagemaker-fmbench-read-121797993273",
    "scripts_prefix": "scripts",
    "script_files": [
      "hf_token.txt"
    ],
    "source_data_prefix": "source_data",
    "tokenizer_prefix": "tokenizer",
    "prompt_template_dir": "prompt_template",
    "prompt_template_file": "prompt_template.txt"
  },
  "run_steps": {
    "0_setup.ipynb": true,
    "1_generate_data.ipynb": true,
    "2_deploy_model.ipynb": false,
    "3_run_inference.ipynb": true,
    "4_model_metric_analysis.ipynb": t

In [5]:
date_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [6]:
## getting access to the s3 bucket where endpoints.json for different models resides
s3_client = boto3.client('s3')

### Access the deployed model endpoints from the endpoints.json file 

In [7]:
## Refer to the file path for the endpoint
## getting the endpoint as an s3 object from the deployed path
try:
    endpoint_info_list = json.loads(get_s3_object(config['aws']['bucket'], ENDPOINT_LIST_PATH))
    logger.info(f"found information for {len(endpoint_info_list)} endpoints in bucket={config['aws']['bucket']}, key={ENDPOINT_LIST_PATH}")
    logger.info(json.dumps(endpoint_info_list, indent=2))
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'NoSuchKey':
        logger.warning(f"Key {ENDPOINT_LIST_PATH} not found in bucket {config['aws']['bucket']}. Using an empty list for endpoints for bedrock models.")
        endpoint_info_list = []
    else:
        raise e

INFO:__main__:found information for 0 endpoints in bucket=sagemaker-fmbench-write-121797993273, key=fmbench-claude-ab3/data/models/endpoints.json
INFO:__main__:[]


In [8]:
# List down the endpoint names that have been deployed
endpoint_name_list = [e['endpoint']['EndpointName'] for e in endpoint_info_list]
logger.info(f"there are {len(endpoint_name_list)} deployed endpoint(s), endpoint_name_list->{endpoint_name_list}")

INFO:__main__:there are 0 deployed endpoint(s), endpoint_name_list->[]


### Creating functions to define and calculate metrics during the time of invocations

In [9]:
def safe_sum(l: List) -> Union[int, float]:
    return sum(filter(None, l))

def safe_div(n: Union[int, float], d: Union[int, float]) -> Optional[Union[int, float]]:
    return n/d if d else None

## Represents the function to calculate all of the metrics at the time of inference
def calculate_metrics(responses, chunk, elapsed_async, experiment_name, concurrency, payload_file) -> Dict:
    
    ## calculate errors based on the completion status of the inference prompt
    errors = [r for r in responses if r['completion'] is None]
    
    ## Calculate the difference as the successes 
    successes = len(chunk) - len(errors)
    
    ## Count all of the prompts token count during inference
    all_prompts_token_count = safe_sum([r['prompt_tokens'] for r in responses])
    prompt_token_throughput = round(all_prompts_token_count / elapsed_async, 2)
    prompt_token_count_mean = safe_div(all_prompts_token_count, successes)
    all_completions_token_count = safe_sum([r['completion_tokens'] for r in responses])
    completion_token_throughput = round(all_completions_token_count / elapsed_async, 2)
    completion_token_count_mean = safe_div(all_completions_token_count, successes)
    transactions_per_second = round(successes / elapsed_async, 2)
    transactions_per_minute = int(transactions_per_second * 60)
    
    ## calculate the latency mean utilizing the safe_sum function defined above
    latency_mean = safe_div(safe_sum([r['latency'] for r in responses]), successes)
    
    ## Function returns all these values at the time of the invocations
    return {
        'experiment_name': experiment_name,
        'concurrency': concurrency,
        'payload_file': payload_file,
        'errors': errors,
        'successes': successes,
        'error_rate': len(errors)/len(chunk),
        'all_prompts_token_count': all_prompts_token_count,
        'prompt_token_count_mean': prompt_token_count_mean,
        'prompt_token_throughput': prompt_token_throughput,
        'all_completions_token_count': all_completions_token_count,
        'completion_token_count_mean': completion_token_count_mean,
        'completion_token_throughput': completion_token_throughput,
        'transactions': len(chunk),
        'transactions_per_second': transactions_per_second,
        'transactions_per_minute': transactions_per_minute,
        'latency_mean': latency_mean
    }

### Set a blocker function and a series of asynchronous concurrent model prompt invocations

In [10]:
def set_metrics(endpoint_name=None,
                    prompt=None,
                    inference_params=None,
                    completion=None,
                    prompt_tokens=None,
                    completion_tokens=None,
                    latency=None) -> Dict:
    return dict(endpoint_name=endpoint_name,                
                prompt=prompt,
                **inference_params,
                completion=completion,
                prompt_tokens=prompt_tokens,
                completion_tokens=completion_tokens,
                latency=latency)

def get_inference(predictor, payload) -> Dict:
    
    latency = 0

    ## initializing completion tokens
    completion_tokens = None
    prompt_tokens = None

    try:
        # get inference      
        resp = predictor.get_prediction(payload) 
        logger.info(f"response={resp}")
        response_json = resp['response_json']
        logger.info(f"response_json={response_json}")
        latency = resp['latency']
        logger.info(f"latency={latency}")
        prompt_tokens = resp['prompt_tokens']
        logger.info(f"prompt_tokens={prompt_tokens}")
        completion_tokens = resp['completion_tokens']
        logger.info(f"completion_tokens={completion_tokens}")

        # Assign the generated_text value to completion
        completion = response_json.get("generated_text", "")

        # Set metrics and logging for both cases
        response = set_metrics(predictor.endpoint_name,
                               payload['inputs'],
                               payload['parameters'],
                               completion,
                               prompt_tokens, 
                               completion_tokens,
                               latency)
        
        # logger.info(f"get_inference, done, endpoint={predictor.endpoint_name}, response={json.dumps(response, indent=2)}, latency={latency:.2f}")
        logger.info(f"get_inference, done, endpoint={predictor.endpoint_name}, prompt_tokens = {prompt_tokens}, completion_tokens={completion_tokens}, latency={latency:.4f}")
    except Exception as e:
        print(f"error occurred with {predictor.endpoint_name}, exception={str(e)}")
        response = set_metrics(predictor.endpoint_name,
                               payload['inputs'],
                               payload['parameters'],
                               None,
                               prompt_tokens,
                               None,
                               None)

    return response

### Setting a series of asynchronous functions to invoke and run inferences concurrently and asynchronously

In [11]:
## Represents a function to start invoking models in separate thread asynchronously for the blocker function
async def async_get_inference(predictor, payload: Dict) -> Dict:
    return await asyncio.to_thread(get_inference, predictor, payload)

## Gathers all of the tasks and sets of the concurrent calling of the asychronous invocations
async def async_get_all_inferences(predictor, payload_list: List) -> List:
    return await asyncio.gather(*[async_get_inference(predictor, payload) for payload in payload_list])

In [12]:
## This function runs the asynchronous function series above together for different experiments and concurrency levels.
async def run_inferences(predictor: sagemaker.base_predictor.Predictor, chunk: List, experiment: Dict, concurrency: int, payload_file: str) -> Tuple[List, Dict]:
    logger.info(f"processing chunk with concurrency={concurrency}")
    s = time.perf_counter()
    responses = await async_get_all_inferences(predictor, chunk)
    elapsed_async = time.perf_counter() - s

    # Add more metadata about this experiment
    for r in responses:
        r['experiment_name'] = experiment['name']
        r['concurrency'] = concurrency

    metrics = calculate_metrics(responses, chunk, elapsed_async, experiment['name'], concurrency, payload_file)
    return responses, metrics

In [13]:
## Function to create the predictors from the experiment we are iterating over
def create_predictor_for_experiment(experiment: Dict, config: Dict, endpoint_info_list: List) -> Optional[sagemaker.base_predictor.Predictor]:

    ## initializing the inference_spec
    inference_spec = None

    ep_info = [e for e in endpoint_info_list if e['experiment_name'] == experiment['name']]
    logger.info(f"endpoint info found is: {ep_info}")

    if ep_info != []:
        ep_name = ep_info[0]['endpoint']['EndpointName']
        inference_spec = experiment.get("inference_spec")
        logger.info(f"experiment name={experiment['name']}, ep_name={ep_name}, inference_spec={inference_spec}")

    elif (ep_info == []) and (experiment['inference_script'] == 'bedrock_predictor.py'):
        ep_name = experiment['ep_name']
        logger.info(f"experiment name={experiment['name']}, custom bring your own ep_name={ep_name}")

    else:
        logger.error(f"endpoint for experiment={experiment['name']} not found, skipping")
        return None

    # create predictor objects
    # Proceed with deployment as before
    # Assuming fmbench is a valid Python package and scripts is a subdirectory within it
    scripts_dir = Path(pkg_resources.files('fmbench'), 'scripts')
    logger.info(f"Using fmbench.scripts directory: {scripts_dir}")

    # Ensure the scripts directory exists
    scripts_dir.mkdir(parents=True, exist_ok=True)
    module_name = Path(experiment['inference_script']).stem
    logger.info(f"script provided for inference from this model is --> {module_name}")
    script_path = scripts_dir / f"{module_name}.py"
    logger.info(f"script path is --> {script_path}")

    # Check and proceed with local script
    if not script_path.exists():
        logger.error(f"script {script_path} not found.")
        return None

    logger.info(f"Deploying using local code: {script_path}")

    spec = importlib.util.spec_from_file_location(module_name, str(script_path))
    inference_module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = inference_module
    spec.loader.exec_module(inference_module)
    # create a predictor from each endpoint in experiments
    return inference_module.create_predictor(ep_name, inference_spec)

endpoint_info_list

[]

In [14]:
## Here, we will process combinations of concurrency levels, the payload files and then loop through the 
## different combinations to make payloads splitted in terms of the concurrency metric and how we can run 
## it and make inference

def create_payload_dict(jline: str, experiment: Dict) -> Dict:
    payload: Dict = json.loads(jline)
    return payload
    
    
def create_combinations(experiment: Dict) -> List[Tuple]:
    combinations_data = []

    # Repeat for each concurrency level
    combinations = list(itertools.product(experiment['concurrency_levels'], experiment['payload_files']))
    logger.info(f"there are {len(combinations)} combinations of {combinations} to run")

    for concurrency, payload_file in combinations:
        # Construct the full S3 file path
        s3_file_path = os.path.join(PROMPTS_DIR, payload_file)
        logger.info(f"s3 path where the payload files are being read from -> {s3_file_path}")

        # Read the payload file from S3
        try:
            response = s3_client.get_object(Bucket=config['aws']['bucket'], Key=s3_file_path)
            payload_file_content = response['Body'].read().decode('utf-8')

            # Create a payload list by processing each line
            payload_list = [create_payload_dict(jline, experiment) for jline in payload_file_content.splitlines()]
            logger.info(f"read from s3://{config['aws']['bucket']}/{s3_file_path}, contains {len(payload_list)} lines")

        except Exception as e:
            logger.error(f"Error reading file from S3: {e}")
            continue

        logger.info(f"creating combinations for concurrency={concurrency}, payload_file={payload_file}, payload_list length={len(payload_list)}")
        
        n = concurrency
        
        if len(payload_list) < n:
            elements_to_add = n - len(payload_list)
            element_to_replicate = payload_list[0]
            # payload_list = payload_list.extend([element_to_replicate]*elements_to_add)
            payload_list.extend([element_to_replicate]*elements_to_add)
            
        # Split the original list into sublists which contain the number of requests we want to send concurrently        
        payload_list_splitted = [payload_list[i * n:(i + 1) * n] for i in range((len(payload_list) + n - 1) // n )]  
        
        for p in payload_list_splitted:
            if len(p) < n:
                elements_to_add = n - len(p)
                element_to_replicate = p[0]
                # p = p.extend([element_to_replicate]*elements_to_add)
                p.extend([element_to_replicate]*elements_to_add)
            

        # Only keep lists that have at least concurrency number of elements
        len_before = len(payload_list_splitted)
        payload_list_splitted = [p for p in payload_list_splitted if len(p) == concurrency]
        logger.info(f"after only retaining chunks of length {concurrency}, we have {len(payload_list_splitted)} chunks, previously we had {len_before} chunks")
        combinations_data.append((concurrency, payload_file, payload_list_splitted))
    logger.info(f"there are {len(combinations)} for {experiment}")
    return combinations_data


In [15]:
# for each experiment
#   - for each endpoint and concurrency in an experiment


def clear_dir(dir_path: str):
    files = glob.glob(os.path.join(dir_path, "*"))
    for f in files:
        os.remove(f)


_ = list(map(clear_dir, [METRICS_PER_INFERENCE_DIR, METRICS_PER_CHUNK_DIR]))

# Initializing the total model instance cost to 0
total_model_instance_cost: int = 0

## To keep track of the cost for all model endpoints
cost_data = []

## To keep track of the experiment durations and the time it takes for the model endpoint to be in service to calculate cost association
experiment_durations = []  

## start the timer before the start of inferences
current_time = datetime.now(timezone.utc)
logger.info(f"Current time recorded while running this experiment is {current_time}..... deployed models are going to start inferences...")

num_experiments: int = len(config['experiments'])
for e_idx, experiment in enumerate(config['experiments']):
    e_idx += 1  # Increment experiment index
    experiment_start_time = time.perf_counter()  # Start timer for the experiment

    predictor = create_predictor_for_experiment(experiment, config, endpoint_info_list)
    if predictor is None:
        logger.error(f"predictor could not be created for experiment={experiment}, moving to next...")
        continue

    combination_data = create_combinations(experiment)

    for concurrency, payload_file, split_payload in combination_data:
        for chunk_index, chunk in enumerate(split_payload):
            logger.info(f"e_idx={e_idx}/{num_experiments}, chunk_index={chunk_index+1}/{len(split_payload)}")

            responses, metrics = await run_inferences(predictor, chunk, experiment, concurrency, payload_file)
            if metrics:
                metrics_json = json.dumps(metrics, indent=2)
                metrics_file_name = f"{time.time()}.json"
                metrics_s3_path = os.path.join(METRICS_PER_CHUNK_DIR, metrics_file_name)
                write_to_s3(metrics_json, config['aws']['bucket'], "", METRICS_PER_CHUNK_DIR, metrics_file_name)

            if responses:
                for r in responses:
                    response_json = json.dumps(r, indent=2)
                    response_file_name = f"{time.time()}.json"
                    response_s3_path = os.path.join(METRICS_PER_INFERENCE_DIR, response_file_name)
                    write_to_s3(response_json, config['aws']['bucket'], "", METRICS_PER_INFERENCE_DIR, response_file_name)
    
    ## initializing the experiment cost
    exp_cost = 0
    
    # Experiment done, stopping the timer for this given experiment
    experiment_end_time = time.perf_counter()

    # calculating the duration of this given endpoint inference time
    experiment_duration = experiment_end_time - experiment_start_time
    logger.info(f"the {experiment['name']} ran for {experiment_duration} seconds......")

    # calculating the per second cost for this instance type
    exp_instance_type: str = experiment['instance_type']
    
    #cost for this given exp
    logger.info(f"metrics json is: {metrics}")
    exp_cost = predictor.calculate_cost(exp_instance_type, config, experiment_duration, metrics)
    logger.info(f"the rate for running {experiment['name']} running on {exp_instance_type} for {experiment_duration} is ${exp_cost}....")

    ## tracking the total cost
    total_model_instance_cost += exp_cost

    experiment_durations.append({
        'experiment_name': experiment['name'],
        'instance_type': exp_instance_type, 
        'duration_in_seconds': f"{experiment_duration:.2f}", 
        'cost': f"{exp_cost:.2f}", 
    })

    logger.info(f"experiment={e_idx}/{num_experiments}, name={experiment['name']}, duration={experiment_duration:.2f} seconds, done")

# experiment_durations.append({'total_cost': f"${total_model_instance_cost:.2f}"})

# After all experiments are done, summarize and optionally save experiment durations along with costs
df_durations = pd.DataFrame(experiment_durations)
logger.info(f"experiment durations: {df_durations}")

# Convert the DataFrame to CSV and write it to S3 or wherever you prefer
csv_buffer_cost = io.StringIO()
df_durations.to_csv(csv_buffer_cost, index=False)
experiment_associated_cost = csv_buffer_cost.getvalue()

# Assuming write_to_s3() is already defined and configured correctly
write_to_s3(experiment_associated_cost, config['aws']['bucket'], "", METRICS_DIR, SUMMARY_MODEL_ENDPOINT_COST_PER_INSTANCE)
logger.info(f"Summary for cost of instance per endpoint per run saved to s3://{config['aws']['bucket']}/{METRICS_DIR}/{SUMMARY_MODEL_ENDPOINT_COST_PER_INSTANCE}")

logger.info(f"total cost of all experiments: ${sum(df_durations.cost.astype(float))}")




INFO:__main__:Current time recorded while running this experiment is 2024-03-23 00:17:14.668818+00:00..... deployed models are going to start inferences...
INFO:__main__:endpoint info found is: []
INFO:__main__:experiment name=claude-instant-v1, custom bring your own ep_name=anthropic.claude-instant-v1
INFO:__main__:Using fmbench.scripts directory: /Users/madhurpt/Desktop/foundation-model-benchmarking-tool-5/src/fmbench/scripts
INFO:__main__:script provided for inference from this model is --> bedrock_predictor
INFO:__main__:script path is --> /Users/madhurpt/Desktop/foundation-model-benchmarking-tool-5/src/fmbench/scripts/bedrock_predictor.py
INFO:__main__:Deploying using local code: /Users/madhurpt/Desktop/foundation-model-benchmarking-tool-5/src/fmbench/scripts/bedrock_predictor.py
INFO:bedrock_predictor:__init__ self._predictor=<botocore.client.BedrockRuntime object at 0x28da7bfd0>
INFO:__main__:there are 1 combinations of [(1, 'payload_en_1000-2000.jsonl')] to run
INFO:__main__:s3

In [16]:
# List .json files in the specified S3 directory
s3_files = list_s3_files(config['aws']['bucket'], METRICS_PER_INFERENCE_DIR)

# Read and parse each JSON file from S3
json_list = list(map(lambda key: json.loads(get_s3_object(config['aws']['bucket'], key)), \
                     s3_files))

# Create DataFrame
df_responses = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_responses.shape} from all responses")
df_responses.head()


INFO:fmbench.utils:found 15 items in bucket=sagemaker-fmbench-write-121797993273, prefix=fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17/per_inference, suffix=.json
INFO:fmbench.utils:there are total of 15 items in bucket=sagemaker-fmbench-write-121797993273, prefix=fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17/per_inference, suffix=.json
INFO:__main__:created dataframe of shape (15, 10) from all responses


,endpoint_name,prompt,ContentType,Accept,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency
0,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,"No, Rhosgoch and Qaleh-Ye Sahar are not locat...",1339,61,1.540054,claude-instant-v1,1
1,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,There is no information provided in the passa...,1932,51,2.289986,claude-instant-v1,1
2,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,"Yes, Gut (Crișul Alb) and Gepiș are both loca...",1154,34,1.727199,claude-instant-v1,1
3,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,"Yes, both Howard Lake (British Columbia) and ...",1646,58,3.230953,claude-instant-v1,1
4,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,"Based on the passages provided, Puka Rumi and...",1397,97,1.895931,claude-instant-v1,1


In [17]:
# List .json files in the specified S3 directory
s3_files = list_s3_files(config['aws']['bucket'], METRICS_PER_CHUNK_DIR)

# Read and parse each JSON file from S3
json_list = list(map(lambda key: json.loads(get_s3_object(config['aws']['bucket'], key)), \
                     s3_files))

# Create DataFrame
df_metrics = pd.DataFrame(json_list)
logger.info(f"created dataframe of shape {df_metrics.shape} from all responses")
df_metrics.head()

INFO:fmbench.utils:found 15 items in bucket=sagemaker-fmbench-write-121797993273, prefix=fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17/per_chunk, suffix=.json
INFO:fmbench.utils:there are total of 15 items in bucket=sagemaker-fmbench-write-121797993273, prefix=fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17/per_chunk, suffix=.json
INFO:__main__:created dataframe of shape (15, 16) from all responses


,experiment_name,concurrency,payload_file,errors,successes,error_rate,all_prompts_token_count,prompt_token_count_mean,prompt_token_throughput,all_completions_token_count,completion_token_count_mean,completion_token_throughput,transactions,transactions_per_second,transactions_per_minute,latency_mean
0,claude-instant-v1,1,payload_en_1000-2000.jsonl,[],1,0.0,1339,1339.0,794.93,61,61.0,36.21,1,0.59,35,1.540054
1,claude-instant-v1,1,payload_en_1000-2000.jsonl,[],1,0.0,1932,1932.0,821.98,51,51.0,21.70,1,0.43,25,2.289986
2,claude-instant-v1,1,payload_en_1000-2000.jsonl,[],1,0.0,1154,1154.0,656.37,34,34.0,19.34,1,0.57,34,1.727199
3,claude-instant-v1,1,payload_en_1000-2000.jsonl,[],1,0.0,1646,1646.0,501.29,58,58.0,17.66,1,0.30,18,3.230953
4,claude-instant-v1,1,payload_en_1000-2000.jsonl,[],1,0.0,1397,1397.0,715.24,97,97.0,49.66,1,0.51,30,1.895931


In [18]:
if endpoint_info_list:
    df_endpoints = pd.json_normalize(endpoint_info_list)
    df_endpoints['instance_type'] = df_endpoints['endpoint_config.ProductionVariants'].map(lambda x: x[0]['InstanceType'])
    cols_for_env = [c for c in df_endpoints.columns if 'Environment' in c]
    print(cols_for_env)
    cols_of_interest = ['experiment_name',
                        'instance_type',
                        'endpoint.EndpointName',
                        'model_config.ModelName',
                        'model_config.PrimaryContainer.Image',
                        'model_config.PrimaryContainer.ModelDataSource.S3DataSource.S3Uri']
    cols_of_interest.extend(cols_for_env)

    df_endpoints = df_endpoints[cols_of_interest]
    cols_of_interest_renamed = [c.split('.')[-1] for c in cols_of_interest]
    df_endpoints.columns = cols_of_interest_renamed
else:
    # Create an empty DataFrame with the desired columns
    df_endpoints = pd.DataFrame(columns=['experiment_name',
                                         'instance_type',
                                         'EndpointName',
                                         'ModelName',
                                         'Image',
                                         'S3Uri'])

# Check if 'experiment_name' column exists in both DataFrames
print("Columns in df_responses:", df_responses.columns)
print("Columns in df_endpoints:", df_endpoints.columns)

df_results = pd.merge(left=df_responses, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')

for e, experiment in enumerate(config['experiments']):
    experiment_name = experiment['name']
    instance_type = experiment['instance_type']
    
    logger.info(f"the instance type: {instance_type}")
    # Update the instance_type column in df_results where the EndpointName matches
    df_results.loc[df_results['experiment_name'] == experiment_name, 'instance_type'] = instance_type

# Inspect the result
df_results.head()

INFO:__main__:the instance type: ClaudeInstant-ODT


Columns in df_responses: Index(['endpoint_name', 'prompt', 'ContentType', 'Accept', 'completion',
       'prompt_tokens', 'completion_tokens', 'latency', 'experiment_name',
       'concurrency'],
      dtype='object')
Columns in df_endpoints: Index(['experiment_name', 'instance_type', 'EndpointName', 'ModelName',
       'Image', 'S3Uri'],
      dtype='object')


,endpoint_name,prompt,ContentType,Accept,completion,prompt_tokens,completion_tokens,latency,experiment_name,concurrency,instance_type,EndpointName,ModelName,Image,S3Uri
0,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,"No, Rhosgoch and Qaleh-Ye Sahar are not locat...",1339,61,1.540054,claude-instant-v1,1,ClaudeInstant-ODT,NaN,NaN,NaN,NaN
1,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,There is no information provided in the passa...,1932,51,2.289986,claude-instant-v1,1,ClaudeInstant-ODT,NaN,NaN,NaN,NaN
2,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,"Yes, Gut (Crișul Alb) and Gepiș are both loca...",1154,34,1.727199,claude-instant-v1,1,ClaudeInstant-ODT,NaN,NaN,NaN,NaN
3,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,"Yes, both Howard Lake (British Columbia) and ...",1646,58,3.230953,claude-instant-v1,1,ClaudeInstant-ODT,NaN,NaN,NaN,NaN
4,anthropic.claude-instant-v1,<s>[INST] <<SYS>>\nYou are an assistant for qu...,application/json,application/json,"Based on the passages provided, Puka Rumi and...",1397,97,1.895931,claude-instant-v1,1,ClaudeInstant-ODT,NaN,NaN,NaN,NaN


In [19]:
# Convert df_results to CSV and write to S3
csv_buffer = io.StringIO()
df_results.to_csv(csv_buffer, index=False)
csv_data_results = csv_buffer.getvalue()
results_file_name = config['report']['per_inference_request_file'].format(datetime=date_time)
results_s3_path = os.path.join(METRICS_DIR, results_file_name)
logger.info(f"results s3 path for per inference csv --> {results_s3_path}")
write_to_s3(csv_data_results, config['aws']['bucket'], "", METRICS_DIR, results_file_name)
logger.info(f"saved results dataframe of shape={df_results.shape} in s3://{BUCKET_NAME}/{results_s3_path}")

INFO:__main__:results s3 path for per inference csv --> fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17/per_inference_request_results.csv
INFO:__main__:saved results dataframe of shape=(15, 15) in s3://sagemaker-fmbench-write-121797993273/fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17/per_inference_request_results.csv


In [20]:
# Ensure the metadata directory exists
os.makedirs(METADATA_DIR, exist_ok=True)

# Path for the metrics_path.txt file
metrics_path_file = os.path.join(METADATA_DIR, 'metrics_path.txt')
logger.info(f"the metrics metadata path is saved here --> {metrics_path_file}")

# Write the METRICS_DIR to metrics_path.txt
with open(metrics_path_file, 'w') as file:
    file.write(METRICS_DIR)

## Write this data to S3
write_to_s3(METRICS_DIR, config['aws']['bucket'], "", DATA_DIR, 'metrics_path.txt')

logger.info(f"the information on the defined path for results on these metrics are given in this --> {METRICS_DIR}")

INFO:__main__:the metrics metadata path is saved here --> metadata/metrics_path.txt
INFO:__main__:the information on the defined path for results on these metrics are given in this --> fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17


In [21]:
# logger.info(f"df_metrics cols = {df_metrics.columns}")
# logger.info(f"df_endpoints cols = {df_endpoints.columns}")
# df_metrics = pd.merge(left=df_metrics, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')

# for e, experiment in enumerate(config['experiments']):
#     experiment_name = experiment['name']
#     instance_type = experiment['instance_type']
    
#     logger.info(f"the instance type: {instance_type}")
#     # Update the instance_type column in df_results where the EndpointName matches
#     df_metrics.loc[df_results['experiment_name'] == experiment_name, 'instance_type'] = instance_type

# df_metrics.head()

# # Convert df_metrics to CSV and write to S3
# csv_buffer = io.StringIO()
# df_metrics.to_csv(csv_buffer, index=False)
# csv_data_metrics = csv_buffer.getvalue()
# metrics_file_name = config['report']['all_metrics_file'].format(datetime=date_time)
# metrics_s3_path = os.path.join(METRICS_DIR, metrics_file_name)
# logger.info(f"results s3 path for metrics csv --> {metrics_s3_path}")
# write_to_s3(csv_data_metrics, config['aws']['bucket'], "", METRICS_DIR, metrics_file_name)
# logger.info(f"saved metrics results dataframe of shape={df_metrics.shape} in s3://{config['aws']['bucket']}/{metrics_s3_path}")

logger.info(f"df_metrics cols = {df_metrics.columns}")
logger.info(f"df_endpoints cols = {df_endpoints.columns}")
df_metrics = pd.merge(left=df_metrics, right=df_endpoints, how='left', left_on='experiment_name', right_on='experiment_name')
for e, experiment in enumerate(config['experiments']):
    experiment_name = experiment['name']
    instance_type = experiment['instance_type']
    
    logger.info(f"the instance type: {instance_type}")
    # Update the instance_type column in df_results where the EndpointName matches
    df_metrics.loc[df_results['experiment_name'] == experiment_name, 'instance_type'] = instance_type
df_metrics.head()

# Convert df_metrics to CSV and write to S3
csv_buffer = io.StringIO()
df_metrics.to_csv(csv_buffer, index=False)
csv_data_metrics = csv_buffer.getvalue()
metrics_file_name = config['report']['all_metrics_file'].format(datetime=date_time)
metrics_s3_path = os.path.join(METRICS_DIR, metrics_file_name)
logger.info(f"results s3 path for metrics csv --> {metrics_s3_path}")
write_to_s3(csv_data_metrics, config['aws']['bucket'], "", METRICS_DIR, metrics_file_name)
logger.info(f"saved metrics results dataframe of shape={df_metrics.shape} in s3://{config['aws']['bucket']}/{metrics_s3_path}")

INFO:__main__:df_metrics cols = Index(['experiment_name', 'concurrency', 'payload_file', 'errors', 'successes',
       'error_rate', 'all_prompts_token_count', 'prompt_token_count_mean',
       'prompt_token_throughput', 'all_completions_token_count',
       'completion_token_count_mean', 'completion_token_throughput',
       'transactions', 'transactions_per_second', 'transactions_per_minute',
       'latency_mean'],
      dtype='object')
INFO:__main__:df_endpoints cols = Index(['experiment_name', 'instance_type', 'EndpointName', 'ModelName',
       'Image', 'S3Uri'],
      dtype='object')
INFO:__main__:the instance type: ClaudeInstant-ODT
INFO:__main__:results s3 path for metrics csv --> fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17/all_metrics.csv
INFO:__main__:saved metrics results dataframe of shape=(15, 21) in s3://sagemaker-fmbench-write-121797993273/fmbench-claude-ab3/data/metrics/yyyy=2024/mm=03/dd=22/hh=20/mm=17/all_metrics.csv
